In [1]:
import time
import os
import pandas as pd
from datetime import datetime, timedelta
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

In [2]:
provinces = ['BKK','Chiangmai','Khonkaen','Rayong','Saraburi','Surat']
train_file_names = ['_train','_temp_surface','_weather_wind']
train_data_paths = []
train_data_dict = {}
lat_lon_dict = {'BKK':[13.729984,100.536443],'Chiangmai':[18.840633,98.969661],'Khonkaen':[16.445329,102.835251],'Rayong':[12.671521,101.275875],'Saraburi':[14.685833,100.871996],'Surat':[9.126057,99.325355]}

In [17]:
for province in provinces:
    train_data_paths.append('./'+province+'/train/')
    train_data_dict[province] = []
    for file_name in train_file_names:
        train_data_dict[province].append(train_data_paths[-1]+province.lower()+file_name+'.csv')

print(train_data_dict)

{'BKK': ['./BKK/train/bkk_train.csv', './BKK/train/bkk_temp_surface.csv', './BKK/train/bkk_weather_wind.csv'], 'Chiangmai': ['./Chiangmai/train/chiangmai_train.csv', './Chiangmai/train/chiangmai_temp_surface.csv', './Chiangmai/train/chiangmai_weather_wind.csv'], 'Khonkaen': ['./Khonkaen/train/khonkaen_train.csv', './Khonkaen/train/khonkaen_temp_surface.csv', './Khonkaen/train/khonkaen_weather_wind.csv'], 'Rayong': ['./Rayong/train/rayong_train.csv', './Rayong/train/rayong_temp_surface.csv', './Rayong/train/rayong_weather_wind.csv'], 'Saraburi': ['./Saraburi/train/saraburi_train.csv', './Saraburi/train/saraburi_temp_surface.csv', './Saraburi/train/saraburi_weather_wind.csv'], 'Surat': ['./Surat/train/surat_train.csv', './Surat/train/surat_temp_surface.csv', './Surat/train/surat_weather_wind.csv']}


In [40]:
for province in provinces:
    province_train = pd.read_csv(train_data_dict[province][0])
    province_temp_serface = pd.read_csv(train_data_dict[province][1])
    province_wheather_wind = pd.read_csv(train_data_dict[province][2])
    province_train = province_train.rename(columns={"Unnamed: 0": "date_time"})
    province_train['date_time'] = pd.to_datetime(province_train['date_time'])
    province_temp_serface['date_time'] = pd.to_datetime(province_temp_serface['date_time'])
    province_wheather_wind['date_time'] = pd.to_datetime(province_wheather_wind['date_time'])
    province_merge_temp_wind = pd.merge(province_temp_serface, province_wheather_wind, on='date_time')
    province_merge = pd.merge(province_train, province_merge_temp_wind, on='date_time',how='left')
    province_merge = province_merge.fillna(method='ffill')
    province_merge = province_merge.fillna(method='bfill')
    province_merge['lat'] = lat_lon_dict[province][0]
    province_merge['long'] = lat_lon_dict[province][1]
    province_merge.to_csv('./'+province+'/train/'+province.lower()+'_train_format.csv',index=False)